In [ ]:
from eComply.eComply import eComply
from ParksGIS.ParksGIS import GISFactory, Server, LayerServerGen, LayerQuery, LayerEdits
from pandas import DataFrame
# from Transformer import Transformer

Filters

In [ ]:
from datetime import datetime
from typing import Any, Callable, Literal, Optional, final
from pandas import merge, Series


def to_json(obj: Any) -> str:
    return obj.to_json(orient="records", date_format="iso")


def apply_edits(dict: dict) -> dict:
    if dict["edits"] is not None and 0 < len(dict["edits"]):
        try:
            result = dict["server"].apply_edits(dict["edits"])
            print(f"Edits Applied: {len(result)}")
        except Exception as e:
            print(e)

    return dict


def guard_edits(
    dict: dict,
    key: str,
    func: Callable[[dict], Any],
) -> dict:
    edits = dict.get("edits", [])
    if 0 == len(edits):
        return dict

    edits = edits[0]
    ids = edits[~edits[key].isna()][key]
    if 0 == len(ids):
        return dict

    try:
        func(dict)
    except Exception as e:
        print(e)
    finally:
        return dict


def extract_changes(
    repo: Any,
    serverGen: int,
    layerId: int,
    strategy: Callable[[Any, Any], Optional[DataFrame]],
) -> dict[str, Any]:
    dict = {"edits": [], "serverGen": serverGen}

    try:
        changes = repo.extractChanges([LayerServerGen(layerId, serverGen)])
        dict["serverGen"] = changes["layerServerGens"][0]["serverGen"] // 1000

        objectIds = (
            changes["edits"][0]["objectIds"]["adds"]
            + changes["edits"][0]["objectIds"]["updates"]
        )
        if 0 < len(objectIds):
            result = strategy(repo, objectIds)
            if result is not None:
                dict["edits"].append(result)
        else:
            print(f"No changes from layer {layerId}")
    except Exception as e:
        print(e)
    finally:
        return dict


def get_changes(
    serverGen: int,
    strategy: Callable[[datetime], LayerEdits],
) -> dict[str, Any]:
    dict = {"edits": []}
    try:
        result = strategy(datetime.fromtimestamp(serverGen))
        if result is not None:
            dict["edits"].append(result)
    except Exception as e:
        print(e)
    finally:
        return dict


def get_edits(dict: dict) -> DataFrame:
    return dict["edits"][0]


def set_edits(dict: dict, data: DataFrame):
    dict["edits"][0] = data


def join(
    list: Series | list,
    guid: bool = False,
) -> str:
    if guid:
        return "'" + "','".join(str(i) for i in list) + "'"
    else:
        return ",".join(str(i) for i in list)


#######################################################################################


# Sending
def contract_extract_changes(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 1

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    ["*"],
                    f"OBJECTID IN ({join(objectIds)}) AND EcomplyContract = 1",
                )
            ]
        )[layerId]

        print(f"Contracts Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict = extract_changes(repo, serverGen, layerId, strategy)
    dict["server"] = repo
    dict["service"] = service
    return dict


def contract_send_edits(dict: dict) -> dict:
    def strategy(dict):
        edits = get_edits(dict)
        # print(to_json(edits))

        dict["service"].postContracts(to_json(edits))
        print(f"Contracts Sent: {dict['edits']}")

    return guard_edits(dict, "OBJECTID", strategy)


def work_order_extract_changes(
    repo: Server,
    serverGen: int,
    service: eComply,
) -> dict:
    layerId = 0

    def strategy(repo, objectIds):
        changes = repo.query(
            [
                LayerQuery(
                    layerId,
                    [
                        "InspectionGlobalID",
                        "Type",
                        "Status",
                        "LocationDetails",
                        "ActualFinishDate",
                        "Comments",
                        "Contract",
                        "CancelReason",
                        "GlobalID",
                        "ClosedDate",
                        "ClosedByERN",
                        "ClosedByName",
                        "CancelDate",
                        "CancelByERN",
                        "CancelByName",
                        "CreatedDate",
                        "CreatedBYERN",
                        "CreatedByName",
                        "UpdatedDate",
                        "UpdatedByERN",
                        "UpdatedByName",
                        "WOEntity",
                        "PROJSTARTDATE",
                        "Project",
                        "RecommendedSpecies",
                        "ClosedBySystem",
                        "OBJECTID",
                    ],
                    f"OBJECTID IN ({join(objectIds)})",
                )
            ]
        )[layerId].rename(
            columns={
                "LocationDetails": "Location",
                "GlobalID": "WorkOrderGlobalId",
                "PROJSTARTDATE": "ProjStartDate",
                "RecommendedSpecies": "RecSpecies",
                "OBJECTID": "ObjectId",
            }
        )

        print(f"Work Orders Extracted: {len(changes)}")
        # print(to_json(changes[:1]))
        return changes

    dict = extract_changes(repo, serverGen, layerId, strategy)
    dict["server"] = repo
    dict["service"] = service
    return dict


def wo_get_associated_planting_space_globalid(dict: dict) -> dict:
    key = "InspectionGlobalID"

    def strategy(dict):
        layerId = 4

        edits = get_edits(dict)
        inspections = (
            dict["server"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        ["PlantingSpaceGlobalID", "GlobalID"],
                        f"GlobalID IN ({join(edits[~edits[key].isna()][key], True)})",
                    )
                ]
            )[layerId]
            .rename(columns={"GlobalID": key})
        )

        edits = merge(edits, inspections, on=key, how="left")
        del edits[key]

        set_edits(dict, edits)
        # print(to_json(edits[:1]))
        print(f"Planting Space Ids found: {len(inspections)}")

    return guard_edits(dict, key, strategy)


def wo_get_associated_planting_space(dict: dict) -> dict:
    key = "PlantingSpaceGlobalID"

    def strategy(dict):
        layerId = 2

        edits = get_edits(dict)
        plantingSpaces = (
            dict["server"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "ParkName",
                            "ParkZone",
                            "Borough",
                            "CommunityBoard",
                            "BuildingNumber",
                            "StreetName",
                            "CityCouncil",
                            "StateAssembly",
                            "GISPROPNUM",
                            "CrossStreet1",
                            "CrossStreet2",
                            "PlantingSpaceOnStreet",
                            "ObjectID",
                            "GlobalID",
                        ],
                        f"GlobalID IN ({join(edits[~edits[key].isna()][key], True)})",
                    )
                ]
            )[layerId]
            .rename(
                columns={
                    "GlobalID": "PlantingSpaceGlobalID",
                    "OBJECTID": "PlantingSpaceId",
                    "PlantingSpaceOnStreet": "OnStreetSite",
                }
            )
        )

        edits = merge(edits, plantingSpaces, on=key, how="left")

        set_edits(dict, edits)
        # print(to_json(edits[:1]))
        print(f"Planting Spaces hydrated: {len(plantingSpaces)}")

    return guard_edits(dict, key, strategy)


def work_order_send_edits(dict: dict) -> dict:
    def strategy(dict):
        edits = get_edits(dict)
        # print(to_json(edits))

        dict["service"].postWorkOrders(to_json(edits))
        print(f"Work Orders Sent: {len(dict['edits'])}")

    return guard_edits(dict, "PlantingSpaceGlobalID", strategy)


# Receiving
def work_order_get_changes(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 0

    def strategy(fromDateTime: datetime):
        changes = service.get_work_orders(fromDateTime)
        apply_edits = LayerEdits(layerId, updates=changes)
        print(f"WorkOrders Recieved: {len(changes)}")
        return apply_edits

    dict = get_changes(serverGen, strategy)
    dict["server"] = repo

    return dict


def wo_update_associated_inspection(dict: dict) -> dict:
    layerId = 4
    key = "WorkOrderGlobalID"

    def strategy(dict):
        edits = get_edits(dict)

        inspections = (
            dict["server"]
            .query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "InspectionGlobalID",
                            "HasActiveWorkOrder",
                        ],
                        f"{key} IN ({join(edits[key], True)})",
                    )
                ]
            )[layerId]
            .rename(columns={"PlantingSpaceGlobalID": "plantingSpaceGlobalId"})
        )

        edits = merge(inspections, edits, on=key, how="left")
        edits.loc[
            edits["Status"] == Literal["Closed", "Canceled"], "HasActiveWorkOrder"
        ] = 0

        dict["edits"].append(
            LayerEdits(
                layerId, updates=edits["InspectionGlobalID", "HasActiveWorkOrder"]
            )
        )

        print(f"Inspections To Update: {len(inspections)}")

    return guard_edits(dict, key, strategy)


def wo_update_associated_platingSpace(dict: dict) -> dict:
    layerId = 2

    def strategy(dict):
        edits = get_edits(dict)

        plantingSpaces = (
            dict["server"].query(
                [
                    LayerQuery(
                        layerId,
                        [
                            "GlobalID",
                            "BuildingNumber",
                            "StreetName",
                            "CrossStreet1",
                            "CrossStreet2",
                        ],
                        f"GlobalID IN ({join(edits['plantingSpaceGlobalId'], True)})",
                    )
                ]
            )[layerId]
            # .rename(
            #     columns={
            #         "GlobalID": "plantingSpaceGlobalId",
            #     }
            # )
        )

        # Transformer.update(
        #     plantingSpaces,
        #     edits,
        #     "plantingSpaceGlobalId",
        #     {
        #         "BuildingNumber": {"Source": "BuildingNumber"},
        #         "StreetName": {"Source": "StreetName"},
        #         "CrossStreet1": {"Source": "CrossStreet1"},
        #         "CrossStreet2": {"Source": "CrossStreet2"},
        #     },
        # )

        dict["edits"].append(LayerEdits(layerId, updates=plantingSpaces))

        print(f"Planting Spaces To Update: {len(plantingSpaces)}")

    return guard_edits(dict, "plantingSpaceGlobalId", strategy)


def contract_get_changes(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 1

    def strategy(fromDateTime):
        changes = service.get_contracts(fromDateTime)
        apply_edits = LayerEdits(layerId, updates=changes)
        print(f"Contracts Recieved: {len(changes)}")
        return apply_edits

    dict = get_changes(serverGen, strategy)
    dict["server"] = repo

    return dict


def work_order_line_item_get_changes(
    service: eComply,
    repo: Server,
    serverGen: int,
) -> dict:
    layerId = 2

    def strategy(fromDateTime):
        changes = service.get_work_order_line_items(fromDateTime)
        apply_edit = LayerEdits(layerId, updates=changes)
        print(f"Line Items Recieved: {len(changes)}")
        return apply_edit

    dict = get_changes(serverGen, strategy)
    dict["server"] = repo

    return dict

Pipeline

In [ ]:
proxy = "@bcpxy.nycnet:8080"

import os

# Set proxy incase environment not set
os.environ["HTTP_PROXY"] = proxy
os.environ["HTTPS_PROXY"] = proxy
# bypass proxy on parks domains
os.environ["NO_PROXY"] = ".parks.nycnet"

# arcGIS = "https://formsgisportal.parks.nycnet"
# arcGIS = 'https://stg-formsgisportal.parks.nycnet'
arcGIS = "https://dev-formsgisportal.parks.nycnet"

factory = GISFactory(
    url=arcGIS + "/portal/home",
    username="forms.python_user",
    password="formsPython24*",
)
eComplyRepo = factory.create_feature(
    arcGIS + "/server/rest/services/eComply/eComplyContract/FeatureServer"
)
dataPushRepo = factory.create_feature(
    arcGIS + "/server/rest/services/DataPush/ForMSDataPush/FeatureServer"
)

service = eComply(
    url="https://nycparks-stage.ecomply.us/WebAPI",
    username="ff@ecomply.us",
    password="!test123",
)

from toolz import pipe

# Push DomainValues
dataPushRepo = factory.create_feature(arcGIS + 'server/rest/services/DataPush/ForMSDataPush/FeatureServer')
domainValues = dataPushRepo.query_domains([LayerDomainNames(0, ['WOContract','WOProject','WOEntity','WOStatus','WOCategory','WOType','GenusSpecies'])])
domains = [
    {
        'domainName': domain['name'],
        'code': str(value['code']),
        'value': value['name'],
    }
    for domain in domainValues for value in domain['codedValues']
]
service.post_domain_values(domains)
print('DomainValues Pushed')

import time

seconds = 60 * 60 * 24
epoch = time.time() - seconds
serverGens = DataFrame({"Contract": [epoch], "WorkOrder": [epoch]})

# serverGens = eComplyRepo.query([LayerQuery(3, ["*"])])[3]
newServerGens = serverGens.copy()
print(newServerGens)

sendContracts = pipe(
    contract_extract_changes(eComplyRepo, serverGens.at[0, 'Contract'], service),
    contract_send_edits,
)
newServerGens['Contract'] = sendContracts['serverGen']
print(newServerGens)
eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

sendWorkOrders = pipe(
    work_order_extract_changes(dataPushRepo, serverGens.at[0, "WorkOrder"], service),
    wo_get_associated_planting_space_globalid,
    wo_get_associated_planting_space_data,
    work_order_send_edits,
)
newServerGens['WorkOrder'] = sendWorkOrders['serverGen']
print(newServerGens)
eComplyRepo.apply_edits([LayerEdits(3, updates=newServerGens.to_json(orient="records"))])

applyWorkOrders = pipe(
    work_order_get_changes(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
    wo_update_associated_inspection,
    wo_update_associated_platingSpace,
    apply_edits
)

applyContracts = pipe(
    contract_get_changes(service, eComplyRepo, serverGens.at[0, 'Contract']),
    apply_edits
)

applyWorkOrderLineItems = pipe(
    work_order_line_item_get_changes(service, dataPushRepo, serverGens.at[0, 'WorkOrder']),
    apply_edits
)